In [1]:
from itertools import chain
import json
from lib.legiscan import legiscan_api, locate_matches
from mergedeep import merge
from operator import itemgetter
import pandas as pd
import re
from typing import Dict, Iterable, Set, Tuple

from lib.legiscan import infer_resolver_map, augment_resolver_map

LEGISCAN_API_URL = 'https://api.legiscan.com/'
ACLU_DATA_PATH = '../datasets/aclu.json'
TTL_DATA_PATH = '../datasets/tracktranslegislation.json'
RESOLVER_MAP = '../artifacts/resolver_map.json'
HINT_MAP = '../configuration/resolver_hints.json'

new_mapper = infer_resolver_map(TTL_DATA_PATH, RESOLVER_MAP, HINT_MAP)
aclu_data = pd.read_json(ACLU_DATA_PATH)

augment_resolver_map(
    new_mapper, 
    [(row['state']['value'], row['name']) for idx, row in aclu_data.iterrows()], #  if row['state']['value']=='ID'
    RESOLVER_MAP,
)

Error: ('OH', 'SB 83') Unknown prefix S for OH SB 83
Error: ('TX', 'HB 5127') No relevant matches for TX HB 5127
Error: ('TX', 'HB 5261') No relevant matches for TX HB 5261
Error: ('MA', 'H. 458') Unknown state MA for MA H. 458
Error: ('MA', 'H. 509') Unknown state MA for MA H. 509
Error: ('TX', 'SB 17') No relevant matches for TX SB 17
Error: ('TX', 'SB 2199') No relevant matches for TX SB 2199
Error: ('TX', 'SB 8') No relevant matches for TX SB 8
Error: ('TX', 'HB 4624') No relevant matches for TX HB 4624
Error: ('TX', 'HB 3164') No relevant matches for TX HB 3164
Error: ('ME', 'HP 577') No relevant matches for ME HP 577
Matching RI HB 5688 with lsid 1712566 RI H5688 (Parents Bill Of Rights Act)
Error: ('ME', 'HP 447') No relevant matches for ME HP 447
Error: ('ID', 'HB 63') No relevant matches for ID HB 63
Error: ('IA', 'HF 9') No relevant matches for IA HF 9
Successfully mapped 412/426 bills (14 not mapped)
Changes persisted
